In [25]:
import requests as rq
import pandas as pd
from tqdm.notebook import tqdm
from datetime import datetime, date

# Сбор свечей


In [2]:
securities_df = pd.read_csv('data/securities')
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   engine               2405 non-null   object 
 1   market               2405 non-null   object 
 2   secid                2405 non-null   object 
 3   isin                 2105 non-null   object 
 4   marketprice_boardid  915 non-null    object 
 5   name                 2405 non-null   object 
 6   emitent_id           1844 non-null   float64
 7   emitent_title        1844 non-null   object 
 8   type                 2405 non-null   object 
 9   group                2405 non-null   object 
dtypes: float64(1), object(9)
memory usage: 188.0+ KB


In [3]:
securities_df.head()

,engine,market,secid,isin,marketprice_boardid,name,emitent_id,emitent_title,type,group
0,stock,shares,ABIO,RU000A0JNAB6,TQBR,"ПАО ""Артген""",1142.0,"Публичное акционерное общество ""Артген биотех""",common_share,stock_shares
1,stock,shares,ABRD,RU000A0JS5T7,TQBR,Абрау-Дюрсо ПАО ао,4787.0,"Публичное акционерное общество ""Абрау – Дюрсо""",common_share,stock_shares
2,stock,shares,ACKO,RU000A0JXS91,TQBR,АСКО ПАО ао,9314.0,"Публичное акционерное общество ""АСКО""",common_share,stock_shares
3,stock,shares,AFKS,RU000A0DQZE3,TQBR,"АФК ""Система"" ПАО ао",777.0,"Публичное акционерное общество ""Акционерная фи...",common_share,stock_shares
4,stock,shares,AFLT,RU0009062285,TQBR,Аэрофлот-росс.авиалин(ПАО)ао,1242.0,"Публичное акционерное общество ""Аэрофлот – рос...",common_share,stock_shares


In [30]:
def get_day_candles_df_from_2023(engine,market,security):
    response = rq.get(
        f'https://iss.moex.com/iss/engines/{engine}/markets/{market}/securities/{security}/candles.json'
        ,params={
            'from': '2023-01-01 00:00:00'
            ,'interval': 24
        }
    )
    candles_json = response.json()['candles']
    df = pd.DataFrame(columns=candles_json['columns'],data=candles_json['data'])
    df['date'] = df['begin'].map(lambda x: datetime.strptime(x[0:10],'%Y-%m-%d'))
    df = df.drop(['begin','end'],axis=1)
    return df

get_day_candles_df_from_2023('stock','shares','ABIO')

,open,close,high,low,value,volume,date
0,62.06,62.80,64.32,61.12,18578839.0,294180,2023-01-03
1,62.84,62.86,63.50,62.16,10031622.4,159610,2023-01-04
2,63.26,62.70,63.26,62.16,7119124.4,113310,2023-01-05
3,62.28,62.48,62.94,62.28,5708542.0,91080,2023-01-06
4,62.60,65.90,68.92,62.54,170448999.8,2563020,2023-01-09
...,...,...,...,...,...,...,...
493,75.10,77.20,79.90,74.92,59736909.0,766810,2024-12-09
494,78.10,75.60,78.10,74.64,30603614.4,403890,2024-12-10
495,75.64,75.12,75.70,74.50,11115170.0,148280,2024-12-11
496,75.12,73.50,75.20,73.00,10474324.6,140820,2024-12-12


In [34]:
for i in tqdm(range(len(securities_df))):
# for i in tqdm(range(10)):
    item = securities_df.iloc[i]
    engine,market,security = item['engine'],item['market'],item['secid']
    df = get_day_candles_df_from_2023(engine,market,security)
    df['engine'],df['market'],df['secid'] = engine,market,security
    if i==0:
        df.to_csv('data/candles',mode='w',index=False,header=True)
    else:
        df.to_csv('data/candles',mode='a',index=False,header=False)

  0%|          | 0/2405 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
candle_df = pd.read_csv('data/candles')

In [33]:
candle_df

,open,close,high,low,value,volume,date,engine,market,secid
0,62.06,62.80,64.32,61.12,1.857884e+07,294180,2023-01-03,stock,shares,ABIO
1,62.84,62.86,63.50,62.16,1.003162e+07,159610,2023-01-04,stock,shares,ABIO
2,63.26,62.70,63.26,62.16,7.119124e+06,113310,2023-01-05,stock,shares,ABIO
3,62.28,62.48,62.94,62.28,5.708542e+06,91080,2023-01-06,stock,shares,ABIO
4,62.60,65.90,68.92,62.54,1.704490e+08,2563020,2023-01-09,stock,shares,ABIO
...,...,...,...,...,...,...,...,...,...,...
3080,88.67,89.87,89.87,88.66,2.816229e+04,316,2024-12-09,stock,shares,AKHT
3081,88.94,88.34,89.00,87.52,3.208344e+06,36068,2024-12-10,stock,shares,AKHT
3082,88.63,87.70,88.63,87.16,2.480214e+04,282,2024-12-11,stock,shares,AKHT
3083,87.70,86.20,88.37,85.90,6.315690e+03,72,2024-12-12,stock,shares,AKHT
